## Plotting Conformation Distribution of AF2-Generated Kinases

In [1]:
import pandas as pd
import plotly.express as px

### Read Inputs

In [2]:
#alphafold input files
AF_results = pd.read_csv('inputs/alphafold_predictions.csv') #contains conformation predicted
AF_metadata = pd.read_csv('inputs/AF_metadata.csv') #contains structure metadata

#PDB database input files
PDB_results = pd.read_csv('inputs/PDB_predictions.csv') #contains pdb conformation predicted
PDB_metadata = pd.read_csv('inputs/pdb_metadata.csv') #contains structure metadata

#general input files
kinase_info = pd.read_csv('inputs/kinases_all.csv')  #contains human kinase group/family info

### Color Palette

In [3]:
#color palette
palette = ['#1F77B4','#1F77B4','#FF7F0E','#FF7F0E','#2CA02C','#2CA02C','#D62728','#D62728','#9467BD','#9467BD']

### Create DataFrames of kinase conformation distribution by species & kinase group

#### Method to create dataframes of conformation distribution by species

In [4]:
def create_df_species(df_results, df_metadata):
    df_metadata = pd.merge(df_metadata[['id','Entry','Organism']],df_results[['id','Class']],
                          on='id',how='right')
    df_metadata['Class'] = df_results.apply(lambda x: x.Class.upper(),axis=1)

    #Fix labels
    xlabel = []
    species_label = []
    for idx,row in df_metadata.iterrows():
        organism = row.Organism
        Class = row.Class

        if organism == 'Homo sapiens (Human)':
            species = 'Human'
        else:
            #species = 'All Kinase Structures'
            species = 'All'
        xlabel.append(f'{Class}<br>({species})')
        species_label.append(species)

    df_metadata['species_label'] = species_label
    df_metadata['xlabel'] = xlabel    

    #Count
    df_metadata['count'] = ''
    df_metadata['total'] = ''
    conf_all_species = df_metadata[['count','Class','species_label']].groupby(['Class']).count().reset_index()
    conf_all = df_metadata[['count','Class','species_label']].groupby(['Class','species_label']).count().reset_index()
    conf_all.loc[conf_all[conf_all['species_label'] == 'All'].index,'count'] = conf_all_species['count'].tolist()

    conf_all['total'] = ''
    conf_all.loc[conf_all[conf_all['species_label'] == 'All'].index,'total'] = \
        conf_all[conf_all['species_label'] == 'All']['count'].sum()

    conf_all.loc[conf_all[conf_all['species_label'] == 'Human'].index,'total'] = \
        conf_all[conf_all['species_label'] == 'Human']['count'].sum()

    #fix labels again
    xlabel = []
    for idx,row in conf_all.iterrows():
        species = row.species_label
        Class = row.Class
        xlabel.append(f'{Class}<br>({species})')

    conf_all['xlabel'] = xlabel
    conf_all['percentage'] = conf_all.apply(lambda x: x['count']/x['total']*100,axis=1)
    conf_all['label'] = conf_all.apply(lambda x: str(int(x['percentage']))+'%'+'('+str(x['count'])+')',axis=1)
    
    return df_metadata, conf_all

#### Method to create dataframes of conformation distribution by kinase group

In [5]:
def create_df_group(df_results, df_metadata, kinase_info):
    #keep human kinases to sort by kinase group
    conf_human = df_metadata[df_metadata['species_label'] == 'Human']

    #create dataframe grouped by kinase group
    conf_human = pd.merge(conf_human,kinase_info[['uniprot','group','family']],
                         left_on='Entry',right_on='uniprot').drop('uniprot',axis=1)
    conf_human = conf_human[~conf_human.isnull().any(1)]

    confs = conf_human[['count','Class','group']].groupby(['Class','group']).count().reset_index()
    conf_groups = pd.DataFrame([],index=confs['group'].unique(),columns=['CIDI','CIDO','CODI','CODO','ωCD'])
    for idx,row in conf_groups.iterrows():
        for column in row.index:
            try:
                conf_groups.loc[idx,column] = confs[(confs['Class'] == column) & (confs['group'] == idx)]['count'].values[0]
            except IndexError:
                conf_groups.loc[idx,column] = 0
                
    conf_human['sum']=''
    conf_human = conf_human[conf_human['group'] != 'RGC']
    confs=conf_human[['count','Class','group']].groupby(['Class','group']).count().reset_index()
    kinase_count=conf_human[['sum','group']].groupby(['group']).count().reset_index()
    conf_count=pd.merge(confs,kinase_count,on='group',how='left')
    conf_count['freq'] = conf_count.apply(lambda x: float(x['count'])/float(x['sum']),axis=1)           
    
    return conf_count


### AF2 conformation distribution

In [6]:
AF_results['id'] = AF_results.apply(lambda x: x.pdb_id.split('-')[1],axis=1).tolist()
AF_metadata['id'] = AF_metadata['Entry']

AF_metadata, conf_all_AF = create_df_species(AF_results, AF_metadata)
conf_AF_count = create_df_group(AF_results, AF_metadata, kinase_info)

In [7]:
fig = px.bar(conf_all_AF,
             x='xlabel', y='percentage', text='label', 
             pattern_shape="xlabel", pattern_shape_sequence=["", "/"]*4,
             color='xlabel',color_discrete_sequence=palette)
fig.update_traces(textposition='outside')
fig.update_layout(
    title="Kinase Conformations (AF)",
    xaxis_title="conformations",
    yaxis_title="percentage",
    showlegend=False,
    font=dict(
        size=20,
    ),
    bargap=0,
    uniformtext_minsize=20,  #xaxis_tickangle=45
)

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'kinase_conf_total_PDB',
    'width': 1080,
    'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.update_yaxes(range=[0, 100])
fig.write_html('outputs/AF_conformations.html')
fig.update_layout(legend= {'itemsizing': 'constant','orientation':'h','xanchor':"center",'x':0.5})
fig.write_image('outputs/AF_conformations.png')
fig.show()


In [8]:
fig = px.bar(conf_AF_count,
             x='group', y='freq', text='count',
             color='Class',color_discrete_sequence=px.colors.qualitative.D3,
             facet_col_spacing=0)
fig.update_traces(textangle=0)
fig.update_layout(
    title="Conformations per Kinase Group (AF)",
    xaxis_title="",
    yaxis_title="frequency",
    font=dict(
        size=20,
    ),
    uniformtext_minsize=20, uniformtext_mode='hide'
)
fig.update_layout(legend= {'itemsizing': 'constant','orientation':'h','xanchor':"center",'x':0.5})

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'kinase_conf_perkinase_AF',
    'width': 1080,
    'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.update_traces(width=1,textposition='inside')
fig.write_html('outputs/AF_conformations_group.html')
fig.show(config=config)

### PDB conformation distribution

In [9]:
PDB_results['id'] = PDB_results.apply(lambda x: x.pdb_id.split('_')[0],axis=1).tolist()
PDB_metadata['id'] = PDB_metadata['From']

PDB_metadata, conf_all_PDB = create_df_species(PDB_results, PDB_metadata)
conf_PDB_count = create_df_group(PDB_results, PDB_metadata, kinase_info)

In [10]:
fig = px.bar(conf_all_PDB,
             x='xlabel', y='percentage', text='label', 
             pattern_shape="xlabel", pattern_shape_sequence=["", "/"]*4,
             color='xlabel',color_discrete_sequence=palette)
fig.update_traces(textposition='outside')
fig.update_layout(
    title="Kinase Conformations (RSCB PDB)",
    xaxis_title="conformations",
    yaxis_title="percentage",
    showlegend=False,
    font=dict(
        size=20,
    ),
    bargap=0,
    uniformtext_minsize=20,  #xaxis_tickangle=45
)

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'kinase_conf_total_PDB',
    'width': 1080,
    'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.update_yaxes(range=[0, 100])
fig.write_html('outputs/PDB_conformations.html')
fig.update_layout(legend= {'itemsizing': 'constant','orientation':'h','xanchor':"center",'x':0.5})
fig.write_image('outputs/PDB_conformations.png')
fig.show()


In [11]:
fig = px.bar(conf_PDB_count,
             x='group', y='freq', text='count',
             color='Class',color_discrete_sequence=px.colors.qualitative.D3,
             facet_col_spacing=0)
fig.update_traces(textangle=0)
fig.update_layout(
    title="Conformations per Kinase Group (RCSB PDB)",
    xaxis_title="",
    yaxis_title="frequency",
    font=dict(
        size=20,
    ),
    uniformtext_minsize=20, uniformtext_mode='hide'
)

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'kinase_conf_perkinase_PDB',
    'width': 1080,
    'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.update_layout(legend= {'itemsizing': 'constant','orientation':'h','xanchor':"center",'x':0.5})

fig.update_traces(width=1,textposition='inside')
fig.write_html('outputs/PDB_conformations_group.html')
fig.show(config=config)